<h1><center>Building Applications Powered by LLMs with LangChain</center></h1>
<hr><hr><hr>
<h3>Introduction</h3>
- LangChain is designed to assist developers in building end-to-end applications using language models. It offers an array of tools, components, and interfaces that simplify the process of creating applications powered by large language models and chat models. <br>
- LangChain streamlines managing interactions with LLMs, chaining together multiple components, and integrating additional resources, such as APIs and databases.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Configurations:-
----------------------

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")
embedding_model_deployment_name = os.getenv("AZURE_OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

## Prompt use case:
----------------------
- A key feature of LangChain is its support for prompts, which encompasses:
    1. Prompt management,
    2. Prompt optimization, and
    3. A generic interface for all LLMs.

    The framework also provides common utilities for working with LLMs.

- `ChatPromptTemplate` is used to create a structured conversation with the AI model, making it easier to manage the flow and content of the conversation. In LangChain, message prompt templates are used to construct and work with prompts, allowing us to exploit the underlying chat model's potential fully.

- System and Human prompts differ in their roles and purposes when interacting with chat models. `SystemMessagePromptTemplate` provides initial instructions, context, or data for the AI model, while `HumanMessagePromptTemplate` are messages from the user that the AI model responds to.

- Using the `to_messages` object in LangChain allows you to convert the formatted value of a chat prompt template into a list of message objects. This is useful when working with chat models, as it provides a structured way to manage the conversation and ensures that the chat model can understand the context and roles of the messages.

#### To illustrate it, let’s create a chat-based assistant that helps users find information about movies.

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [6]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.0
)

In [8]:
system_template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_message = chat_prompt.format_prompt(movie_title="Inception").to_messages()
print( chat_message )

[SystemMessage(content='You are an assistant that helps users find information about movies.'), HumanMessage(content='Find information about the movie Inception.')]


In [10]:
response = llm.invoke( chat_message )
print( response.content )

"Inception" is a science fiction action film directed by Christopher Nolan. It was released in 2010 and stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Marion Cotillard. The film follows a professional thief who steals information by infiltrating the subconscious of his targets through their dreams. 

The story revolves around Dom Cobb (played by DiCaprio), who is offered a chance to have his criminal history erased in exchange for performing the act of "inception" - planting an idea in someone's mind rather than stealing it. As Cobb and his team delve deeper into the layers of dreams, they face various challenges and encounter unexpected twists.

"Inception" received critical acclaim for its originality, visual effects, and complex narrative. It won four Academy Awards, including Best Cinematography, Best Sound Editing, Best Sound Mixing, and Best Visual Effects. The film was also a commercial success, grossing over $828 million worldwide.

If you are interes

## Summarization chain example:-
----------------------------------
- LangChain prompts can be found in various use cases, such as summarization or question-answering chains.
- For example, when creating a summarization chain, LangChain enables interaction with an external data source to fetch data for use in the generation step. This could involve summarizing a lengthy piece of text or answering questions using specific data sources.


- The following code will initialize the language model using `AzureChatOpenAI` class with a temperature of 0 - because we want deterministic output.
- The `load_summarize_chain` function accepts an instance of the language model and returns a pre-built summarization chain.
- Lastly, the `PyPDFLoader` class is responsible for loading PDF files and converting them into a format suitable for processing by LangChain. To use it, install `PyPDF` using `pip install pypdf`

In [15]:
pip show pypdf

Name: pypdf
Version: 4.1.0
Summary: A pure-python PDF library capable of splitting, merging, cropping, and transforming PDF files
Home-page: 
Author: 
Author-email: Mathieu Fenniak <biziqe@mathieu.fenniak.net>
License: 
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_openai import AzureChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.0
)

In [20]:
# Load the summarization chain
summarize_chain = load_summarize_chain(llm)

# Ask user to enter the PDF file path that is to be summarised.
pdf_file_path = input("Enter path of PDF file to be summarized: ")

# Load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path=pdf_file_path)
complete_document = document_loader.load()


# Summarize the document
summary = summarize_chain.invoke( complete_document )
print("Summary:-",summary['output_text'], sep="\n")

Enter path of PDF file to be summarized:  E:\Programs & Codes\Generative AI\pdf_docs\AI - PSO.pdf


Summary:-
Particle Swarm Optimization (PSO) is a population-based metaheuristic strategy for solving continuous nonlinear optimization problems. It is inspired by the collective social behaviors of swarm movements, such as birds or fish searching for food. In PSO, particles represent potential solutions, and a swarm is formed by a set of particles initially distributed randomly in a search space. The position of each particle is updated based on the experiences of its neighbors, including itself. Each particle has knowledge of its own fitness value, its locally best solution, and the globally best solution of the swarm. PSO algorithm involves the sharing of information about global fitness. The concepts of memory influence and inertia weight are also discussed in relation to PSO.


## QA chain example:
------------------------
- We can also use LangChain to manage prompts for asking general questions from the LLMs.
- These models are proficient in addressing fundamental inquiries.
- Nevertheless, it is crucial to remain mindful of the potential issue of hallucinations, where the models may generate non-factual information. To address this concern, we will later introduce the Retrieval chain as a means to overcome this problem.

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.0
)

- We define a custom prompt template by creating an instance of the `PromptTemplate` class.
- The template string contains a placeholder `{question}` for the input question, followed by a newline character and the `"Answer:"` label.
- The `input_variables` argument is set to the list of available placeholders in the prompt (like a question in this case) to indicate the name of the variable that the chain will replace in the template `.invoke()` method.

In [6]:
prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

chain = LLMChain(prompt=prompt, llm=llm)

In [10]:
response = chain.invoke( "Explain in a pointwise manner, how can we plot a straight line on a XY plane, passing through 2 points with only coordinates of that 2 points given?" )
print( response["text"] )

1. Identify the coordinates of the two points given. Let's call them (x1, y1) and (x2, y2).

2. Calculate the slope of the line using the formula: slope = (y2 - y1) / (x2 - x1). This formula represents the change in y divided by the change in x between the two points.

3. Determine the y-intercept of the line using the formula: y-intercept = y1 - slope * x1. This formula represents the value of y when x is equal to 0.

4. Use the slope and y-intercept to write the equation of the line in slope-intercept form: y = mx + b, where m is the slope and b is the y-intercept.

5. Plot the two points on the XY plane.

6. Use the equation of the line to find additional points on the line by substituting different x-values into the equation and solving for y.

7. Connect the points on the XY plane to form a straight line passing through the two given points.
